# Web Scraping — Part 1

*Inspired by web scraping lessons from [Lauren Klein](https://github.com/laurenfklein/emory-qtm340/blob/master/notebooks/class4-web-scraping-complete.ipynb) and [Allison Parrish](https://github.com/aparrish/dmep-python-intro/blob/master/scraping-html.ipynb)*

In this series of lessons, we're going to introduce how to "scrape" data from the internet with the Python libraries requests and BeautifulSoup.

We will cover how to:

* Programmatically access the text of a web page
* Understand the basics of HTML
* Extract certain HTML elements

## Why Do We Need To Scrape At All?

To understand the significance of web scraping, let's walk through the likely data collection process behind [“Film Dialogue from 2,000 screenplays, Broken Down by Gender and Age”](https://pudding.cool/2017/03/film-dialogue/).

To find their 2,000 screenplays, Hannah Andersen and Matt Daniels consulted a number of already existing sources — one of which was the [Cornell Movie Dialogues Corpus](http://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html). This is a corpus created by Cornell CIS professors Cristian Danescu-Niculescu-Mizil and Lillian Lee for their paper ["Chameleons in imagined conversations"](http://www.cs.cornell.edu/~cristian/papers/chameleons.pdf). Go Big Red!

These researchers helpfully shared a dataset of every URL that they used to find and access the screenplays in their own project.

**Import Pandas**

In [35]:
import pandas as pd

**Read in CSV file**

In [36]:
urls = pd.read_csv("../data/cornell-movie-corpus/raw_script_urls.csv", delimiter='\t', encoding='utf=8')

**Display DataFrame**

In [37]:
urls

,id,movie_title,script_url
0,m0,10 things i hate about you,http://www.dailyscript.com/scripts/10Things.html
1,m1,1492: conquest of paradise,http://www.hundland.org/scripts/1492-Conquest...
2,m2,15 minutes,http://www.dailyscript.com/scripts/15minutes....
3,m3,2001: a space odyssey,http://www.scifiscripts.com/scripts/2001.txt
4,m4,48 hrs.,http://www.awesomefilm.com/script/48hours.txt
...,...,...,...
612,m612,watchmen,http://www.scifiscripts.com/scripts/wtchmn.txt
613,m613,xxx,http://www.dailyscript.com/scripts/xXx.txt
614,m614,x-men,http://www.scifiscripts.com/scripts/xmenthing...
615,m615,young frankenstein,http://www.horrorlair.com/scripts/young.txt


Each movie title in this CSV file is paired with a URL for the screenplay. How can we actually use these URLs to get computationally tractable text data?

Though we could manually navigate to each URL and copy/paste each screenplay into a file, that would be suuuuper slow and painstaking, and we would lose crucial data in the process — information that might help us automatically distinguish the title of the movie from the screenplay itself, for example. It would be much better to programmatically access the text data attached to every URL.

## Responses and Requests

To programmatically access the text data attached to every URL, we can use a Python library called [requests](https://requests.readthedocs.io/en/master/).

When you type in a URL in your search address bar, you're sending an HTTP **request** for a web page, and the server which stores that web page will accordingly send back a **response**, some web page data that your browser will render.

<img src="images/request-response.png" class="center" width="700" height="400">

### Import Requests 

In [38]:
import requests

### Get HTML Data

With the `.get()` method, we can request to "get" web page data for a specific URL, which we will store in a varaible called `response`.

In [39]:
response = requests.get("http://www.scifiscripts.com/scripts/Ghostbusters.txt")

### HTTP Status Code

If we check out `response`, it will simply tell us its [HTTP response code](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status), aka whether the request was successful or not.

"200" is a successful response, while "404" is a common "Page Not Found" error.

In [40]:
response

<Response [200]>

Let's see what happens if we change the title of the movie from *Ghostbusters* to *Ghostboogers* in the URL...

In [41]:
bad_response = requests.get("http://www.scifiscripts.com/scripts/Ghostboogers.txt")

In [42]:
bad_response

<Response [404]>

<img src="images/Ghostboogers.png" class="center" width="900" height="400">

### Extract Text From Web Page

To actually get at the text data in the reponse, we need to use `.text`, which we will save in a variable called `html_string`. The text data that we're getting is formatted in the HTML markup language, which we will talk more about in the BeautifulSoup section below.

In [43]:
html_string = response.text

Here's the screenplay now in a variable.

In [44]:
print(html_string[:500] + "...")

						Ghostbusters

							by
						Harold Ramis
							and
						Dan Aykroyd

					Final Shooting Script
				Last revised October 7, 1983


											FADE IN

EXT. NEW YORK PUBLIC LIBRARY -- DAY

The sun shines brightly on the classic facade of the main library at Fifth
Avenue and 42nd Street.  In the adjacent park area, pretty hustlers and
drug peddlers go about their business.

FRONT STEPS

A few people lounge on the steps flanked by the familiar stone lions.

INT. MAI...


### Extract Text From Multiple Web Pages

Let's quickly demonstrate how we might extract the screenplay text for every URL in the DataFrame. To do so, we're going to create a smaller DataFrame from the Cornell Movie Dialogue Corpus, which consists of the first 10 movies.

In [45]:
sample_urls = urls[:10]

In [46]:
sample_urls

,id,movie_title,script_url
0,m0,10 things i hate about you,http://www.dailyscript.com/scripts/10Things.html
1,m1,1492: conquest of paradise,http://www.hundland.org/scripts/1492-Conquest...
2,m2,15 minutes,http://www.dailyscript.com/scripts/15minutes....
3,m3,2001: a space odyssey,http://www.scifiscripts.com/scripts/2001.txt
4,m4,48 hrs.,http://www.awesomefilm.com/script/48hours.txt
5,m5,the fifth element,http://www.scifiscripts.com/scripts/5thelemen...
6,m6,8mm,http://www.dailyscript.com/scripts/eight-mill...
7,m7,a nightmare on elm street 4: the dream master,http://www.hundland.org/scripts/A-Nightmare-o...
8,m8,a nightmare on elm street: the dream child,http://www.hundland.org/scripts/A-Nightmare-o...
9,m9,the atomic submarine,http://leonscripts.tripod.com/scripts/ATOMICS...


We're going to make a function called `scrape_screenplay()` that includes our `requests.get()` and `response.text` code.

In [47]:
def scrape_screenplay(url):
    response = requests.get(url) #verify=False
    html_string = response.text
    return html_string

Then we're going apply it to the "script_url" column of the DataFrame and create a new column for the resulting extracted text.

In [48]:
sample_urls = sample_urls.copy()

for index, row in sample_urls.iterrows():
    try:
        sample_urls.loc[index, 'text'] = scrape_screenplay(row['script_url'])
    except:
        sample_urls.loc[index, 'text'] = "URL not available"

In [49]:
sample_urls

,id,movie_title,script_url,text
0,m0,10 things i hate about you,http://www.dailyscript.com/scripts/10Things.html,<html>\r\n<head>\r\n<title>10 Things I Hate Ab...
1,m1,1492: conquest of paradise,http://www.hundland.org/scripts/1492-Conquest...,URL not available
2,m2,15 minutes,http://www.dailyscript.com/scripts/15minutes....,<html>\r\n<head>\r\n<title>15 Minutes script b...
3,m3,2001: a space odyssey,http://www.scifiscripts.com/scripts/2001.txt,Note from poster to Kubrick newsgroup:\r\n\r\n...
4,m4,48 hrs.,http://www.awesomefilm.com/script/48hours.txt,"\n\t\t\t\t\t""48 HRS.""\n\n\t\t\t\t\t by\n\n\t..."
5,m5,the fifth element,http://www.scifiscripts.com/scripts/5thelemen...,\t\t\t\tThe Fifth Element\r\n\r\n\t\t\t\tAn or...
6,m6,8mm,http://www.dailyscript.com/scripts/eight-mill...,"<html>\n\n<head>\n <title>""8MM"", by Andrew K..."
7,m7,a nightmare on elm street 4: the dream master,http://www.hundland.org/scripts/A-Nightmare-o...,URL not available
8,m8,a nightmare on elm street: the dream child,http://www.hundland.org/scripts/A-Nightmare-o...,URL not available
9,m9,the atomic submarine,http://leonscripts.tripod.com/scripts/ATOMICS...,<html>\r\n\r\n<head>\r\n <title>ATOMIC SUBMA...


The DataFrame above is truncated, so we can't see the full contents of the "text" column. But if we print out every row in the column, we can see that we successfully extracted text for each URL (though some of these URLs returned 404 errors).

In [50]:
for text in sample_urls['text']:
    print(text[:500])

<html>
<head>
<title>10 Things I Hate About You script by Karen McCullah Lutz, Kirsten Smith & William Shakespeare</title>
<meta name="description" content="10 Things I Hate About You script by Karen McCullah Lutz, Kirsten Smith & William Shakespeare">
<meta name="keywords" content="10 Things I Hate About You script, 10 Things I Hate About You movie script, 10 Things I Hate About You film script, 10 Things I Hate About You screenplay">
<meta http-equiv="content-type" content="text/html; cha
URL not available
<html>
<head>
<title>15 Minutes script by John Hertzfeld</title>
<meta name="description" content="15 Minutes script by John Hertzfeld">
<meta name="keywords" content="15 Minutes script, 15 Minutes movie script, 15 Minutes film script, 15 Minutes screenplay">
<meta http-equiv="content-type" content="text/html; charset=iso-8859-1">
<meta name="robots" content="index,follow">
<meta http-equiv="Content-Language" content="EN">
<meta name="objecttype" content="Document">
<meta name="Sub

## Web Scraping

Not all web pages will be as easy to scrape as these screenplay files, however. Let's say we wanted to scrape the lyrics for Missy Elliott's song "The Rain (Supa Dupa Fly)" (1997) from Genius.com.

<img src="images/Missy-Elliott.png" class="center" width="700" height="500">

Even at a glance, we can tell that this *Genius* web page is a lot more complicated than the *Ghostbusters* page and that it contains a lot of information beyond the lyrics.

Sure enough, if we use our requests library again and try to grab the data for this web page, the underlying data is much more complicated, too.

In [51]:
response = requests.get("https://genius.com/Missy-elliott-the-rain-supa-dupa-fly-lyrics")
html_string = response.text
print(html_string[:2000] + "...")

<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Genius</title>
    <style>*{box-sizing:border-box}body{width:100%;margin:0;color:#fff;background:#000;font-family:Helvetica Neue,Arial,Helvetica,Geneva,sans-serif;font-size:20px;font-weight:700}a,a:active,a:focus{color:#3d85c6;text-decoration:none}a:hover{border-bottom:1px dotted}header{background-color:#ffff64;padding:10px;color:#000}header img{height:20px;display:block;margin:0 auto}main{max-width:860px;margin:0 auto}.text{text-align:center}.alert{margin:40px 20px}h1{font-size:7.5rem;text-transform:uppercase;font-weight:700;margin:0}@media screen and (max-width:420px){h1{font-size:3.5rem}}.distractions{margin:3rem auto 0;width:100%;max-width:700px}.iframe-container{width:100%;height:0;padding-top:56.25%;position:relative}.iframe-container iframe{max-width:100%;max-height:100

How can we extract just the song lyrics from this messy soup of a document? Luckily there's a Python library that can help us called BeautifulSoup, which parses HTML documents.

To understand BeautifulSoup and HTML, we're going to briefly depart from our Missy Elliot lyrics challenge to consider a much simpler website. This toy website was made by the poet, programmer, and professor Allison Parrish explicitly for the purposes of teaching BeautifulSoup.

## HTML

Parrish's website is titled "Kittens and the TV Shows They Love." It can be found at the following URL: http://static.decontextualize.com/kittens.html

<img src="images/kittens-web.png" class="center" width="800" height="500">

If we use our requests library on this Kittens TV website, this is what we get:

In [52]:
response = requests.get("http://static.decontextualize.com/kittens.html")
html_string = response.text
print(html_string)

<!doctype html>
<html>
	<head>
		<title>Kittens!</title>
		<style type="text/css">
			span.lastcheckup { font-family: "Courier", fixed; font-size: 11px; }
		</style>
	</head>
	<body>
		<h1>Kittens and the TV Shows They Love</h1>
		<div class="kitten">
			<h2>Fluffy</h2>
			<div><img src="kitten1.jpg"></div>
			<ul class="tvshows">
				<li>
					<a href="http://www.imdb.com/title/tt0106145/">Deep Space Nine</a>
				</li>
				<li>
					<a href="http://www.imdb.com/title/tt0088576/">Mr. Belvedere</a>
				</li>
			</ul>
			Last check-up: <span class="lastcheckup">2014-01-17</span>
		</div>
		<div class="kitten">
			<h2>Monsieur Whiskeurs</h2>
			<div><img src="kitten2.jpg"></div>
			<ul class="tvshows">
				<li>
					<a href="http://www.imdb.com/title/tt0106179/">The X-Files</a>
				</li>
				<li>
					<a href="http://www.imdb.com/title/tt0098800/">Fresh Prince</a>
				</li>
			</ul>
			Last check-up: <span class="lastcheckup">2013-11-02</span>
		</div>
	</body>
</html>




### HTML Tags

This is an HTML document. HTML stands for HyperText Markup Language. It is the standard language for writing web page documents. The most important thing you need to know about HTML is that the language uses HTML "tags" to represent different elements, such as a main header `<h1>`. 

| HTML Tag                | Explanation                              |
|--------------------|-------------------------------------------|
| <\!DOCTYPE>        | Defines document type                 |
| <html\>             | Defines HTML document                  |
| <head\>             | Main information about document    |
| <title\>            | Title for document          |
| <body\>             | Document body               |
| <h1\> to <h6\>       |  Headings                    |
| <p\>                | Paragraph                       |
| <br\>               | Line break               |
| <\!\-\-comment here-\-> | Comment                         |
| <img\> | Image                         |
| <a\> | Hyperlink                       |
| <ul\> | Unordered list                     |
| <ol\> | Ordered list                     |
| <li\> | List item                     |
| <style\> | Style information for a document                    |
| <div\> | Section in a document                   |
| <span\> | Section in a document                   |

HTML tags often, but not always, require a "closing" tag. For example, the main header "Kittens and the TV Shows They Love" will be surrounded by `<h1>` (opening tag) and `</h1>` (closing tag) on either side: `<h1>Kittens and the TV Shows They Love</h1>`

### HTML Attributes, Classes, and IDs

HTML elements sometimes come with even more information inside a tag. This will often be a keyword (like `class` or `id`) followed by an equals sign `=` and a further descriptor such as `<div class="kitten">`

We need to know about tags as well as attributes, classes, and IDs because this is how we're going to extract specific HTML data with BeautifulSoup.

## BeautifulSoup

In [53]:
from bs4 import BeautifulSoup

To make a BeautifulSoup document, we call `BeautifulSoup()` with two parameters: the `html_string` from our HTTP request and [the kind of parser](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#specifying-the-parser-to-use) that we want to use, which will always be `"html.parser"` for our purposes.

In [54]:
response = requests.get("http://static.decontextualize.com/kittens.html")
html_string = response.text

document = BeautifulSoup(html_string, "html.parser")

In [55]:
document

<!DOCTYPE html>

<html>
<head>
<title>Kittens!</title>
<style type="text/css">
			span.lastcheckup { font-family: "Courier", fixed; font-size: 11px; }
		</style>
</head>
<body>
<h1>Kittens and the TV Shows They Love</h1>
<div class="kitten">
<h2>Fluffy</h2>
<div><img src="kitten1.jpg"/></div>
<ul class="tvshows">
<li>
<a href="http://www.imdb.com/title/tt0106145/">Deep Space Nine</a>
</li>
<li>
<a href="http://www.imdb.com/title/tt0088576/">Mr. Belvedere</a>
</li>
</ul>
			Last check-up: <span class="lastcheckup">2014-01-17</span>
</div>
<div class="kitten">
<h2>Monsieur Whiskeurs</h2>
<div><img src="kitten2.jpg"/></div>
<ul class="tvshows">
<li>
<a href="http://www.imdb.com/title/tt0106179/">The X-Files</a>
</li>
<li>
<a href="http://www.imdb.com/title/tt0098800/">Fresh Prince</a>
</li>
</ul>
			Last check-up: <span class="lastcheckup">2013-11-02</span>
</div>
</body>
</html>

Once you inspect the HTML and find the image source, like "kitten1.jpg" or "kitten2.jpg" in the example above, you can combine it with the base URL, download the image using requests.get().content, and save it to your directory by writing the content to a file.

In [65]:
img_content = requests.get("http://static.decontextualize.com/kitten1.jpg").content

with open("../data/fluffy.jpg", "wb") as f:
    f.write(img_content)

### Extract HTML Element

We can use the `.find()` method to find and extract certain elements, such as a main header.

In [57]:
document.find("h1")

<h1>Kittens and the TV Shows They Love</h1>

If we want only the text contained between those tags, we can use `.text` to extract just the text.

In [58]:
document.find("h1").text

'Kittens and the TV Shows They Love'

In [59]:
type(document.find("h1").text)

str

Find the HTML element that contains an image.

In [60]:
document.find("img")

<img src="kitten1.jpg"/>

### Extract Multiple HTML Elements

You can also extract multiple HTML elements at a time with `.find_all()`

In [61]:
document.find_all("img")

[<img src="kitten1.jpg"/>, <img src="kitten2.jpg"/>]

In [62]:
document.find_all("div", attrs={"class": "kitten"})

[<div class="kitten">
 <h2>Fluffy</h2>
 <div><img src="kitten1.jpg"/></div>
 <ul class="tvshows">
 <li>
 <a href="http://www.imdb.com/title/tt0106145/">Deep Space Nine</a>
 </li>
 <li>
 <a href="http://www.imdb.com/title/tt0088576/">Mr. Belvedere</a>
 </li>
 </ul>
 			Last check-up: <span class="lastcheckup">2014-01-17</span>
 </div>,
 <div class="kitten">
 <h2>Monsieur Whiskeurs</h2>
 <div><img src="kitten2.jpg"/></div>
 <ul class="tvshows">
 <li>
 <a href="http://www.imdb.com/title/tt0106179/">The X-Files</a>
 </li>
 <li>
 <a href="http://www.imdb.com/title/tt0098800/">Fresh Prince</a>
 </li>
 </ul>
 			Last check-up: <span class="lastcheckup">2013-11-02</span>
 </div>]

In [63]:
document.find_all("h2")

[<h2>Fluffy</h2>, <h2>Monsieur Whiskeurs</h2>]

``` {warning}
Heads up! The code below will cause an error.
```

Let's try to extract the text from all the header2 elements:

In [64]:
document.find_all("h2").text

AttributeError: ResultSet object has no attribute "text". You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

Uh oh. That didn't work! In order to extract text data from multiple HTML elements, we need a `for` loop and some list-building.

In [ ]:
all_h2_headers = document.find_all("h2")

In [ ]:
all_h2_headers

First we will make an empty list called `h2_headers`.

Then `for` each `header` in `all_h2_headers`, we will grab the `.text`, put it into a variable called `header_contents`, then `.append()` it to our `h2_headers` list.

In [ ]:
h2_headers = []
for header in all_h2_headers:
    header_contents = header.text
    h2_headers.append(header_contents)

In [ ]:
h2_headers

How might we transform this for loop into a *list comprehension*?
```
#For Loop
h2_headers = []
for header in all_h2_headers:
    header_contents = header.text
    h2_headers.append(header_contents)
```

**Check out the *list comprehension* answer here**

In [ ]:
h2_headers = [header.text for header in all_h2_headers]
h2_headers

## Inspect HTML Elements with Browser

Most times if you're looking to extract something from an HTML document, it's best to use your "Inspect" capabilities in your web browser. You can hover over elements that you're interested in and find that specific element in the HTML.

<img src="images/inspect.png" class="center" width="1000" height="600">

For example, if we hover over the main header:

<img src="images/inspect-h1.png" class="center" width="700" height="400">

## Your Turn!

### Find the name of **one** of the kittens and then return the text of the name (either "Fluffy" or "Monsieur Whiskers").

To do so, open the web page (http://static.decontextualize.com/kittens.html) and then use your Developer Tools to find the HTML tag associated with the kitten names.

In [ ]:
# Your code here 👇

Ok so now we've learned a little bit about how to use BeautifulSoup to parse HTML documents. So how would we apply what we've learned to extract Missy Elliott lyrics?

https://genius.com/Missy-elliott-the-rain-supa-dupa-fly-lyrics

In [ ]:
response = requests.get("https://genius.com/Missy-elliott-the-rain-supa-dupa-fly-lyrics")
html_str = response.text

document = BeautifulSoup(html_str, "html.parser")

In [ ]:
document.text[:1000] + "..."

### Extract just the song lyrics for Missy Elliott's "The Rain (Supa Dupa Fly)"

What HTML element do we need to "find" to extract the song lyrics?

In [ ]:
# Your code here 👇

### Extract just the song title for Missy Elliott's "The Rain"

What HTML element do we need to "find" to extract the title?

In [ ]:
# Your code here 👇

### Extract the producer for Missy Elliott's "The Rain"

In [ ]:
# Your code here 👇

## Solutions

### Find the name of **one** of the kittens and then return the text of the name (either "Fluffy" or "Monsieur Whiskers").

In [ ]:
h2_headers[1]

### Extract just the song lyrics for Missy Elliott's "The Rain"

In [ ]:
lyrics_tag = document.find_all("div", attrs={"class":"Lyrics__Container-sc-cbcfa1dc-1 dfzvqs"})

lyrics = []
for content in lyrics_tag:
    line = content.get_text(separator=" ")
    lyrics.append(line)

for content in lyrics:
    print(content)

### Extract just the song title for Missy Elliott's "The Rain"

In [ ]:
song_title = document.find('h1').text
print(song_title)

### Extract the producer for Missy Elliott's "The Rain"

In [ ]:
producers_div = document.find_all("div", attrs={"class":"SongHeader-desktop__CreditList-sc-e33a5cab-16 hMCaSG"}) # explain why HeaderCredits__List-wx7h8g-3

for line in producers_div:
    line_content = line.text
    print(line_content)